# Entendiendo direct step multistep
Que para cada horizonte de forecast se entrene un modelo distinto. 

En lugar de utilizar la recursividad para hacer predicciones h horizonte de tiempo a futuro

In [2]:
import random
import lightgbm as lgb
import pandas as pd
from datasetsforecast.m4 import M4, M4Info
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import smape

from mlforecast import MLForecast
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean
from mlforecast.target_transforms import Differences

### 0. Data

In [3]:
group = 'Hourly'
await M4.async_download('data', group=group)
df, *_ = M4.load(directory='data', group=group)
df['ds'] = df['ds'].astype('int')
ids = df['unique_id'].unique()
random.seed(0)
sample_ids = random.choices(ids, k=4)
sample_df = df[df['unique_id'].isin(sample_ids)]
info = M4Info[group]
horizon = info.horizon
valid = sample_df.groupby('unique_id').tail(horizon)
train = sample_df.drop(valid.index)

In [4]:
def avg_smape(df):
    """Computes the SMAPE by serie and then averages it across all series."""
    full = df.merge(valid)
    return (
        evaluate(full, metrics=[smape])
        .drop(columns='metric')
        .set_index('unique_id')
        .squeeze()
    )

In [6]:
# print datos de entrenamiento
train

,unique_id,ds,y
86796,H196,1,11.8
86797,H196,2,11.4
86798,H196,3,11.1
86799,H196,4,10.8
86800,H196,5,10.6
...,...,...,...
325187,H413,956,59.0
325188,H413,957,58.0
325189,H413,958,53.0
325190,H413,959,38.0


### 1. Train
- Entrenar modelo simple.

- La idea es saber cómo se ven los H MODELOS ENTRENADOS con direct forecast como estrategia

In [13]:
# definir modelo
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[1,2,3],
    num_threads=1,
)

In [14]:
# definir horizonte forecast 
horizon = 5

In [15]:
# entrenar. definir el horizonte y así tener un modelo distinto entrenado para cada horizonte
individual_fcst = fcst.fit(train, max_horizon=horizon)

In [19]:
# predecir
individual_preds = individual_fcst.predict(horizon)
individual_preds

,unique_id,ds,LGBMRegressor
0,H196,961,16.668257
1,H196,962,16.539340
2,H196,963,16.268556
3,H196,964,16.315177
4,H196,965,15.763151
5,H256,961,13.774598
6,H256,962,13.853981
7,H256,963,13.998305
8,H256,964,14.327281
9,H256,965,14.651842


### 2. Obtener modelos por detras
Deberían ser 5 modelos, ya que se definió un forecast con 5 steps a futuro

In [22]:
model = fcst.models['LGBMRegressor']
model

LGBMRegressor(random_state=0, verbosity=-1)

In [35]:
model.predict([[1, 2, 3]])

NotFittedError: Estimator not fitted, call fit before exploiting the model.

### OBS:
- Por lo que se ve, para mejorar los tiempo lo que se hace es no entrenar los modelos en el método fit, sino que cuando se HACE PREDICT AL INDICAR CUANTOS HORIZONTES PREDECIR, AHÍ SE ENTRENA EL MODELO
  
- Es lo único que me cuadra cuando quiero obtener el estimador y hacer mis propias predicciones no aparescan H MODELOS y me diga que el único modelo que aparece NO SE HALLA ENTRENADO

### 3. Guardar modelo y ver qué se guarda
Ya que aparece el modelo no entrenado

In [27]:
# guardar modelo
path_model = 'models/fcst_direct/'
fcst.save(path_model)

In [28]:
# abrir el pkl models.pkl para ver qué contiene
model_loaded = pd.read_pickle(path_model + 'models.pkl')
model_loaded

{'LGBMRegressor': [LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1)]}

In [36]:
# predecir con el modelo que predice el primer step
model_loaded['LGBMRegressor'][0].predict([[1,2,3]])

array([11.91985082])

#### CONCLUSIONES
- SE PUEDE VER QUE AL GUARDAR EL MODELO, REALMENTE SI APARECEN LOS 5 MODELOS

- **ENTONCES, CUANDO SE QUIERE CONSULTAR EL MODELO SE VE MAL, PERO REALMENTE ESTÁN ENTRENADOS EN EL MÉTODO FIT**

In [42]:
# para consultar el modelo, lo que falta es utilizar el atributo correcto de la clase. LLAMAR "models_", AGREGANDO EL GUION 
# BAJO AL FINAL. AHÍ SI SE RETORNAN TODOS LOS MODELOS
fcst.models_

{'LGBMRegressor': [LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1),
  LGBMRegressor(random_state=0, verbosity=-1)]}

In [44]:
fcst.models_['LGBMRegressor'][0].predict([[1,2,3]])

array([11.91985082])